In [3]:
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt

# 输入/输出目录
input_dir = "figs/projections/chest/csv"
output_dir = "figs/projections/chest/projections"

os.makedirs(output_dir, exist_ok=True)

# 找到所有 deg_*.csv
csv_files = glob.glob(os.path.join(input_dir, "deg_*.csv"))

for csv_file in csv_files:
    # 读取 CSV
    data = pd.read_csv(csv_file, header=None).to_numpy()
    
    # 生成输出文件名
    base = os.path.basename(csv_file)       # deg_xx.csv
    name = os.path.splitext(base)[0] + ".png"  # deg_xx.png
    out_path = os.path.join(output_dir, name)

    # 保存为灰度图
    plt.imsave(out_path, data, cmap="gray")
    # print(f"Saved {out_path}")

print("✅ All CSV files converted to PNG.")


✅ All CSV files converted to PNG.


In [ ]:
import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


def to_uint8_gray(img):
    """
    将 plt.imread 得到的图像统一转换为灰度 uint8:
    - 若是浮点，范围通常在[0,1]，先缩放到[0,255]
    - 若是 RGB/RGBA，去掉 alpha 并对通道取均值得到灰度
    """
    arr = img
    # 去 alpha 通道
    if arr.ndim == 3 and arr.shape[2] == 4:
        arr = arr[:, :, :3]
    # 转灰度
    if arr.ndim == 3 and arr.shape[2] == 3:
        # 若是float，先不缩放，先转灰度，最后统一缩放到 uint8
        if np.issubdtype(arr.dtype, np.floating):
            gray = arr.mean(axis=2)  # [0,1]
            gray = np.clip(gray * 255.0, 0, 255).astype(np.uint8)
        else:
            # 整型/uint8 直接均值
            gray = arr.mean(axis=2).round().astype(np.uint8)
    elif arr.ndim == 2:
        # 已是灰度
        if np.issubdtype(arr.dtype, np.floating):
            gray = np.clip(arr * 255.0, 0, 255).astype(np.uint8)
        else:
            gray = arr.astype(np.uint8)
    else:
        raise ValueError(f"Unsupported image shape: {arr.shape}")
    return gray


# 输入/输出目录
input_dir = "figs/projections/chest/projections"
output_dir = "figs/projections/chest/mask"
os.makedirs(output_dir, exist_ok=True)

# 找到所有 deg_*.png
png_files = glob.glob(os.path.join(input_dir, "deg_*.png"))

for png_file in png_files:
    # 读取并统一到灰度 uint8
    img = plt.imread(png_file)
    gray = to_uint8_gray(img)

    # 生成 mask：像素为 0 或 255 -> 0，其它 -> 1
    mask = np.where((gray == 0) | (gray == 255), 0, 1).astype(np.uint8)

    # 保存为 CSV（与输入同名）
    base = os.path.basename(png_file)             # deg_xx.png
    name = os.path.splitext(base)[0] + ".csv"     # deg_xx.csv
    out_path = os.path.join(output_dir, name)
    pd.DataFrame(mask).to_csv(out_path, index=False, header=False)

    print(f"Saved mask {out_path}")

print("✅ All mask CSV files generated.")


In [14]:
import os
import re
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# ---------------------------
# Utilities
# ---------------------------
def to_uint8_gray(img):
    """统一转灰度 uint8"""
    arr = img
    if arr.ndim == 3 and arr.shape[2] == 4:  # RGBA -> RGB
        arr = arr[:, :, :3]
    if arr.ndim == 3 and arr.shape[2] == 3:
        if np.issubdtype(arr.dtype, np.floating):
            gray = arr.mean(axis=2)           # [0,1]
            gray = np.clip(gray * 255.0, 0, 255).astype(np.uint8)
        else:
            gray = arr.mean(axis=2).round().astype(np.uint8)
    elif arr.ndim == 2:
        if np.issubdtype(arr.dtype, np.floating):
            gray = np.clip(arr * 255.0, 0, 255).astype(np.uint8)
        else:
            gray = arr.astype(np.uint8)
    else:
        raise ValueError(f"Unsupported image shape: {arr.shape}")
    return gray

def build_initial_mask(gray, mode="threshold", low_ratio=0.00, high_ratio=0.95):
    """
    生成单张图的初始 mask:
    - mode="threshold": 仅保留 [low_ratio, high_ratio] 的像素为 1（其余 0）
    - mode="extreme":   将灰度==0 或 255 的像素置 0，其余 1
    """
    if mode == "threshold":
        low  = int(255 * low_ratio)
        high = int(255 * high_ratio)
        mask = np.where((gray >= low) & (gray <= high), 1, 0).astype(np.uint8)
    elif mode == "extreme":
        mask = np.where((gray == 0) | (gray == 255), 0, 1).astype(np.uint8)
    else:
        raise ValueError("mode must be 'threshold' or 'extreme'")
    return mask

def angle_from_filename(path):
    """从 'deg_XXX.png' 中解析角度整数"""
    m = re.search(r"deg_(\d+)\.png$", os.path.basename(path))
    if not m:
        raise ValueError(f"Bad filename: {path}")
    return int(m.group(1)) % 360

# ---------------------------
# Config
# ---------------------------
input_dir  = "figs/projections/chest/projections"
output_dir = "figs/projections/chest/mask"
os.makedirs(output_dir, exist_ok=True)

MASK_MODE   = "threshold"  # 'threshold' or 'extreme'
LOW_RATIO   = 0.05         # 如需同时屏蔽极暗可设 0.05
HIGH_RATIO  = 0.95         # 屏蔽高亮边缘/伪影

# ---------------------------
# Collect files by angle
# ---------------------------
paths = glob.glob(os.path.join(input_dir, "deg_*.png"))
by_angle = {}
for p in paths:
    ang = angle_from_filename(p)
    by_angle[ang] = p

# ---------------------------
# Process pairs (ang, ang+180)
# ---------------------------
mask_rates = []
visited = set()
for ang in sorted(by_angle.keys()):
    if ang in visited:
        continue
    mate = (ang + 180) % 360
    if mate not in by_angle:
        print(f"Skip angle {ang}: pair {mate} not found.")
        continue

    # 读取两张图并做初始 mask
    img_a = plt.imread(by_angle[ang])
    img_b = plt.imread(by_angle[mate])
    gray_a = to_uint8_gray(img_a)
    gray_b = to_uint8_gray(img_b)

    mask_a0 = build_initial_mask(gray_a, mode=MASK_MODE, low_ratio=LOW_RATIO, high_ratio=HIGH_RATIO)
    mask_b0 = build_initial_mask(gray_b, mode=MASK_MODE, low_ratio=LOW_RATIO, high_ratio=HIGH_RATIO)

    # 关键改动：将 +180° 的初始 mask 水平翻转，使其对齐到 θ 的坐标系
    mask_b0_aligned = np.fliplr(mask_b0)

    # 交集（同一有效像素区域）：θ 坐标系
    # intersect_mask_theta = (mask_a0 & mask_b0_aligned).astype(np.uint8)
    intersect_mask_theta = (mask_a0 | mask_b0_aligned).astype(np.uint8)

    # 保存：θ 用 θ 坐标的交集；θ+180° 用“翻回去”的交集（保持各自原生坐标）
    out_a = os.path.join(output_dir, os.path.splitext(os.path.basename(by_angle[ang]))[0]   + ".csv")
    out_b = os.path.join(output_dir, os.path.splitext(os.path.basename(by_angle[mate]))[0] + ".csv")

    pd.DataFrame(intersect_mask_theta).to_csv(out_a, index=False, header=False)
    pd.DataFrame(np.fliplr(intersect_mask_theta)).to_csv(out_b, index=False, header=False)

    kept = int(intersect_mask_theta.sum())
    total = intersect_mask_theta.size
    print(f"[{ang:03d} & {mate:03d}] kept {kept}/{total} ({kept/total:.2%}) -> {out_a}, {out_b}")
    
    mask_rates.append({
    "angle": ang,
    "mate": mate,
    "kept": kept,
    "total": total,
    "mask_rate": kept / total
    })

    visited.add(ang)
    visited.add(mate)

print("✅ Pairwise-intersection masks (with +180° flip alignment) generated for all available angle pairs.")

df_rates = pd.DataFrame(mask_rates)
out_summary = os.path.join(output_dir, "mask_rates_summary.csv")
df_rates.to_csv(out_summary, index=False)

avg_rate = df_rates["mask_rate"].mean()

print(f"\n📊 average mask rate: {1-avg_rate:.2%} (save to {out_summary})")


[000 & 180] kept 54538/65536 (83.22%) -> figs/projections/chest/mask/deg_00.csv, figs/projections/chest/mask/deg_180.csv
[003 & 183] kept 57812/65536 (88.21%) -> figs/projections/chest/mask/deg_03.csv, figs/projections/chest/mask/deg_183.csv
[006 & 186] kept 59746/65536 (91.17%) -> figs/projections/chest/mask/deg_06.csv, figs/projections/chest/mask/deg_186.csv
[009 & 189] kept 60361/65536 (92.10%) -> figs/projections/chest/mask/deg_09.csv, figs/projections/chest/mask/deg_189.csv
[012 & 192] kept 61074/65536 (93.19%) -> figs/projections/chest/mask/deg_12.csv, figs/projections/chest/mask/deg_192.csv
[015 & 195] kept 61709/65536 (94.16%) -> figs/projections/chest/mask/deg_15.csv, figs/projections/chest/mask/deg_195.csv
[018 & 198] kept 62767/65536 (95.77%) -> figs/projections/chest/mask/deg_18.csv, figs/projections/chest/mask/deg_198.csv
[021 & 201] kept 64228/65536 (98.00%) -> figs/projections/chest/mask/deg_21.csv, figs/projections/chest/mask/deg_201.csv
[024 & 204] kept 65149/65536 (99